In [12]:
import pandas as pd
import json
import re
import csv
import numpy as np
import math
import codecs
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Load Data

In [2]:
df = pd.read_csv('data/out_test.csv')
len(df)

10778

In [18]:
df['subject_list'] = df['subjects'].str.split('|').to_list()

In [19]:
df['subject_list']

0                      [Deutsches Literaturarchiv Marbach]
1        [Burckhardt, Jacob, Deutschland, Historienmale...
2                   [Geschichtsstudium, Burckhardt, Jacob]
3                           [Fitzcarraldo, Herzog, Werner]
4                                      [Rauzzini, Filippo]
                               ...                        
10773           [Winckelmann, Johann Joachim, Archäologie]
10774                                                  NaN
10775                                                  NaN
10776                                                  NaN
10777                                                  NaN
Name: subject_list, Length: 10778, dtype: object

In [20]:
vectorizer = CountVectorizer(min_df=1)
subject_doc_matrix = vectorizer.fit_transform(df.subject_list)

AttributeError: 'list' object has no attribute 'lower'

In [9]:
import umap as umap
import matplotlib.pyplot as plt